# Tutorial 1: Image Classification

In the tutorial, you will learn
- how to use mmclassification to classify images
- how to train a flowers classifier in practice
- how to evaluate your model


## Step 1. Installation
The environment for the classification is as follow
Run on the RTX 2080
cuda=='11.0'
python=='3.7'
pytorch=='1,7,1'
torchvision=='0.8.2'
mmcv=='1.3.14'
mmcls=='0.15.0'


Note that the conda environment named 'open-mmlab' has already been installed before, so you may not need to install it by yourself on this node. The following shows how to install mmclassification from scratch. We Create a conda virtual environment and activate it.
```shell
conda create -n open-mmlab python=3.7 -y
conda activate open-mmlab
conda install pytorch cudatoolkit=11.0 torchvision -c pytorch
```
### Install mmcv
```shell
pip install mmcv
```
### Install build requirements and install MMClassification.
```shell
git clone https://github.com/open-mmlab/mmclassification.git
cd mmclassification
pip install -e .  # or "python setup.py develop"
```

If you run 'python' and 'import mmcls' without returning error messages, then the repo has been installed successfully!

## Step 2. Get data and prepare it before setting config

We get the data from https://www.robots.ox.ac.uk/~vgg/data/flowers/17/index.html. It is a task of flower classification with 1360 images totally. There are 17 categories, and each category has 80 images. Here is the data example. From the example, we can observe that some flowers are not very easy classified easily by human eyes.
![Image example](https://www.robots.ox.ac.uk/~vgg/data/flowers/17/categories.jpg)

### Download data
```
wget https://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz
tar zxvf 17flowers.tgz
mkdir data
mv 17flowers data/flowers
```
rename the '17flowers' to 'flowers', and put the directory into ```mmclassification/data/flowers```


### Split data

To fine-tuning on a customized dataset (i.e flowers), the fastest way is to follow the configuration of [Imagenet](https://github.com/open-mmlab/mmclassification/blob/master/docs/tutorials/finetune.md).


We need to split the data into training set and validation set. The directory should be like that
```
flowers
     |--train
          |--class_0
                |--image_xxxx.jpg
                |--image_xxxx.jpg
          |--class_1
                |--image_xxxx.jpg
                |--image_xxxx.jpg         
          ...
          |--class_16
                |--image_xxxx.jpg
                |--image_xxxx.jpg      
     |--val
          |--class_0
                |--image_xxxx.jpg
                |--image_xxxx.jpg
          |--class_1
                |--image_xxxx.jpg
                |--image_xxxx.jpg         
          ...
          |--class_16
                |--image_xxxx.jpg
                |--image_xxxx.jpg  
    |--meta
        |--train.txt
        |--val.txt
```

We first split the data to the format above. Now the directory is ```mmclassification/data/flowers```
```shell
python split.py
```

Then, we need to generate **meta files** for both training data and validation data, to describle (image_path, image_class), which will be used in the config files. The example of a line in the meta file is:
'class_0/image_0005.jpg 0'
'class_2/image_0195.jpg 2'

We generate the meta files by
```shell
mkdir meta
python generate_meta.py
```


In ```mmclassification/mmcls/datasets```, we follow the file 'imagenet.py', which writes the dataset class of Imagenet, to write a dataset class file 'flowers.py'. Since we follow the data directory of Imagenet above, we just need to copy 'imagenet.py' and re-write the **CLASSES** here.  
```python
@DATASETS.register_module()
class Flowers(BaseDataset):

    IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif')
    CLASSES = [
        'daffodil', 'snowdrop', 'lilyValley', 'bluebell', 'crocys', 'iris', 'tigerlily', 'tulip', 'fritillary', 'sunflower', 'daisy', 'colts foot', 'dandelion', 'cowslip', 'buttercup', 'wind flower', 'pansy'
    ]

    def load_annotations(self):
        if self.ann_file is None:
            folder_to_idx = find_folders(self.data_prefix)
            samples = get_samples(
                self.data_prefix,
                folder_to_idx,
                extensions=self.IMG_EXTENSIONS)
            if len(samples) == 0:
                raise (RuntimeError('Found 0 files in subfolders of: '
                                    f'{self.data_prefix}. '
                                    'Supported extensions are: '
                                    f'{",".join(self.IMG_EXTENSIONS)}'))

            self.folder_to_idx = folder_to_idx
        elif isinstance(self.ann_file, str):
            with open(self.ann_file) as f:
                samples = [x.strip().rsplit(' ', 1) for x in f.readlines()]
        else:
            raise TypeError('ann_file must be a str or None')
        self.samples = samples

        data_infos = []
        for filename, gt_label in self.samples:
            info = {'img_prefix': self.data_prefix}
            info['img_info'] = {'filename': filename}
            info['gt_label'] = np.array(gt_label, dtype=np.int64)
            data_infos.append(info)
        return data_infos
```

We also need to import 'Flower' class in the '\__init__.py'



## Step 3. Set config files
 
Now we are at 'mmclassification/configs/_base_' directory

### set data config
a. In ```_base_/datasets```, we add ```flowers_bs32.py``` to describe the basic config about data.
    Note that we need to claim the data meta path in it like that
```
    data = dict(
    samples_per_gpu=32,
    workers_per_gpu=1,
    train=dict(
        type=dataset_type,
        data_prefix='data/flowers/train',
        ann_file='data/flowers/meta/train.txt',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_prefix='data/flowers/val',
        ann_file='data/flowers/meta/val.txt',
        pipeline=test_pipeline),
    test=dict(
        # replace `data/val` with `data/test` for standard test
        type=dataset_type,
        data_prefix='data/flowers/val',
        ann_file='data/flowers/meta/val.txt',
        pipeline=test_pipeline))
    evaluation = dict(interval=1, metric='accuracy')
```

### set model config
b. In ```_base_/models```, we add ```resnet18_flowers.py``` to describe the basic config about model.
```
    type='ImageClassifier',
    backbone=dict(
        type='ResNet',
        depth=18,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=17,
        in_channels=512,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
    ))

```
Note we use the 'resnet-18' model, and set 'num_classes' as 17 since the number of flower categories is 17.

### set training config
c. In ```_base_/schedules```, we add ```flowers_bs32.py``` to describle the basic config about schedule. Here, we set the SGD optimizer with initial learning rate 0.02, and step learning scheduler, which decreases the learning rate (default 10x smaller) at 100th epoch and 150th epoch only.
```
    # optimizer, modified from cifar10_bs128.py
    optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)
    optimizer_config = dict(grad_clip=None)
    # learning policy
    lr_config = dict(policy='step', step=[100, 150])
    runner = dict(type='EpochBasedRunner', max_epochs=200)
```

### set saving config
d. In the ```_base_/default_runtime.py```, we set the config about checkpoint saving and log file.
```
    # checkpoint saving
    checkpoint_config = dict(interval=1)
    # yapf:disable
    log_config = dict(
        interval=100,
        hooks=[
            dict(type='TextLoggerHook'),
            # dict(type='TensorboardLoggerHook')
        ])
    # yapf:enable

    dist_params = dict(backend='nccl')
    log_level = 'INFO'
    load_from = None
    resume_from = None
    workflow = [('train', 1)]
```

And then, we new a config file in ```mmclassification/configs/resnet/resnet18_flowers_bs128.py``` 
we add these config .py files into this file as
```
_base_ = [
    '../_base_/models/resnet18_flowers.py', '../_base_/datasets/flowers_bs32.py',
    '../_base_/schedules/flowers_bs32.py', '../_base_/default_runtime.py'
]
```



## Step 4. train

In the directory of 'mmclassification', we run
```shell
python tools/train.py \
  --config 'configs/resnet/resnet18_flowers_bs128.py' \
  --work-dir 'output/resnet18_flowers_bs128'
```
NOTE: should run in the 'open-mmlab' conda environment!

Here we need to indicate the ```config``` 'configs/resnet/resnet18_flowers_bs128.py', and ```work_dir``` is used for model and log file saving. In this case, the trained model and log are saved in ```output/resnet18_flowers_bs128```. Now we change directory to ```output/resnet18_flowers_bs128```, we are supposed to get saved model and log files. 
The generated log is as follows, the classifiation accuacy arrives 87.6% at 200th epoch.

In [3]:
!python tools/train.py \
  --config 'configs/resnet/resnet18_flowers_bs128.py' \
  --work-dir 'output/resnet18_flowers_bs128'

/userhome/cs2/ad37li5/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
2022-09-24 14:56:11,108 - mmcls - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.13 (default, Mar 29 2022, 02:18:16) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.0, V11.0.221
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.1
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73

## Step 5.  Evaluation

### Demo
Before demo, you need mmcv-full.
```shell
pip install mmcv-full
```

Here we choose one image from the validation set ```image_0005.jpg``` and put it in the ```demo``` directory. And we run this following command to classify this image by our trained model, which is saved at ```output/resnet18_flowers_bs128/```.

The image we choose is 
![Demo image](demo/image_0005.jpg)


Run the model!
```shell
python demo/image_demo.py \
  --img 'demo/image_0005.jpg'\ 
  --config 'configs/resnet/resnet18_flowers_bs128.py' \
  --checkpoint 'output/resnet18_flowers_bs128/epoch_199.pth'
```
Example output:
{'pred_label': 0, 'pred_score': 0.9722172021865845, 'pred_class': 'daffodil'}

From the output, the trained model successfully classify the demo image as 'daffodil' with over 97% confidence.



In [7]:
!python demo/image_demo.py --img 'demo/image_0005.jpg' --config 'configs/resnet/resnet18_flowers_bs128.py' --checkpoint 'output/resnet18_flowers_bs128/epoch_199.pth'

/userhome/cs2/ad37li5/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
load checkpoint from local path: output/resnet18_flowers_bs128/epoch_199.pth
{
    "pred_label": 0,
    "pred_score": 0.9965479969978333,
    "pred_class": "daffodil"
}


### Test 
You can test the trained model by running the following command
```shell
python tools/test.py \
  --config 'configs/resnet/resnet18_flowers_bs128.py' \
  --checkpoint 'output/resnet18_flowers_bs128/epoch_199.pth' \
  --out 'output/resnet18_flowers_bs128/test.json'
```
The output file will be saved in the ```--out```.



In [9]:
!python tools/test.py \
  --config 'configs/resnet/resnet18_flowers_bs128.py' \
  --checkpoint 'output/resnet18_flowers_bs128/epoch_199.pth' \
  --out 'output/resnet18_flowers_bs128/test.json'

/userhome/cs2/ad37li5/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
load checkpoint from local path: output/resnet18_flowers_bs128/epoch_199.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 170/170, 121.0 task/s, elapsed: 1s, ETA:     0s
dumping results to output/resnet18_flowers_bs128/test.json


In [13]:
import json
file = open('output/resnet18_flowers_bs128/test.json')
test_json = json.load(file)
print(json.dumps(test_json, indent=4))

{
    "class_scores": [
        [
            0.28845837712287903,
            7.948243001010269e-05,
            0.00031274688080884516,
            2.19324829231482e-05,
            8.16048850538209e-06,
            0.0034240921959280968,
            1.0076349099108484e-05,
            0.6315582990646362,
            0.0006850928184576333,
            0.00013765037874691188,
            3.086902324866969e-07,
            1.0835431112354854e-06,
            0.0013413811102509499,
            0.07299347221851349,
            0.00014103377179708332,
            6.148808040506992e-08,
            0.0008266650256700814
        ],
        [
            0.8116847276687622,
            3.3150548972571414e-08,
            2.7500023591642275e-08,
            3.688708005711305e-08,
            2.796155285977875e-06,
            0.001953461207449436,
            1.9210445145745325e-07,
            0.18492913246154785,
            2.1231217317563278e-07,
            0.000361483107553795,
        